# Corrections

## Correction exercice ACP

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import decomposition, preprocessing

data = pd.read_csv("billets.csv", sep=";")
data = data.dropna()

standard_scaler = preprocessing.StandardScaler()

billets_data = standard_scaler.fit_transform(data[data.columns[1:]])
pca = decomposition.PCA()
billets_pca = pca.fit_transform(billets_data)
billetspca = pd.DataFrame(billets_pca, index=data.index, columns=["F"+str(i+1) for i in range(6)])
billetspca

,F1,F2,F3,F4,F5,F6
0,1.627035,0.807294,2.192500,3.328620,0.328611,0.002057
1,-2.017133,-2.217975,-1.011861,0.043402,0.043670,-0.468112
2,-0.996369,2.596334,1.241833,-0.878880,-0.868951,0.184537
3,-1.379560,-1.809941,0.594839,0.890183,0.805023,-0.213295
4,0.138409,-0.597355,0.976431,-1.737647,1.147098,-0.295813
...,...,...,...,...,...,...
1495,1.415019,-0.260032,0.912233,0.600323,-0.236258,-1.315906
1496,3.261672,1.330325,0.601857,0.367258,-0.106580,-0.546738
1497,1.878843,-0.599267,-0.532782,-0.117704,-0.297130,0.497393
1498,1.775458,0.506379,0.067710,-0.483507,0.344075,0.523804


In [2]:
pca.explained_variance_ratio_

array([0.43123527, 0.16970651, 0.13136423, 0.11792169, 0.0962036 ,
       0.0535687 ])

## Correction exercice Pycaret

Vous allez à présent détecter de faux billets !

Vous aurez à disposition un jeu de données contenant différentes variables :

1. **is_genuine** : target, nous dit si le billet est un vrai billet ou non.
2. **toutes les autres variables sont des tailles exprimées en cm**

1. Préparation des données

In [3]:
import pandas as pd
data = pd.read_csv("billets.csv", sep=";")
data 

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.46,103.36,103.66,3.77,2.99,113.09
2,True,172.69,104.48,103.50,4.40,2.94,113.16
3,True,171.36,103.91,103.94,3.62,3.01,113.51
4,True,171.73,104.28,103.46,4.04,3.48,112.54
...,...,...,...,...,...,...,...
1495,False,171.75,104.38,104.17,4.42,3.09,111.28
1496,False,172.19,104.63,104.44,5.27,3.37,110.97
1497,False,171.80,104.01,104.12,5.51,3.36,111.95
1498,False,172.06,104.28,104.06,5.17,3.46,112.25


2. Installez et Importez Pycaret 

In [4]:
from pycaret.classification import *

3. Créez un environnement Pycaret

In [5]:
X = data.drop("is_genuine", axis=1)
y = data.is_genuine

session = setup(data=X, target=y, normalize=True, train_size=0.8, data_split_stratify=True, fold=2, session_id=0)

,Description,Value
0,Session id,0
1,Target,is_genuine
2,Target type,Binary
3,Original data shape,"(1500, 7)"
4,Transformed data shape,"(1500, 7)"
5,Transformed train set shape,"(1200, 7)"
6,Transformed test set shape,"(300, 7)"
7,Numeric features,6
8,Rows with missing values,2.5%
9,Preprocess,True


4. Comparez les modèles et sélectionnez le meilleur

In [6]:
from pycaret.classification import compare_models
top_models = compare_models( sort="Accuracy")
print(top_models)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:13:32
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9908,0.9986,0.9962,0.9901,0.9931,0.9793,0.9794,1.1950
nb,Naive Bayes,0.9908,0.9991,0.9950,0.9913,0.9931,0.9793,0.9794,0.8200
svm,SVM - Linear Kernel,0.9908,0.0000,0.9950,0.9913,0.9931,0.9793,0.9794,0.8600
qda,Quadratic Discriminant Analysis,0.9908,0.9991,0.9950,0.9913,0.9931,0.9793,0.9794,0.0200
et,Extra Trees Classifier,0.9908,0.9992,0.9962,0.9901,0.9931,0.9793,0.9794,0.0950
rf,Random Forest Classifier,0.9900,0.9992,0.9950,0.9901,0.9925,0.9774,0.9775,0.1150
knn,K Neighbors Classifier,0.9892,0.9912,0.9950,0.9888,0.9919,0.9756,0.9756,0.8700
gbc,Gradient Boosting Classifier,0.9892,0.9985,0.9938,0.9900,0.9919,0.9756,0.9756,0.0700
ada,Ada Boost Classifier,0.9875,0.9984,0.9912,0.9900,0.9906,0.9719,0.9719,0.0550
lda,Linear Discriminant Analysis,0.9875,0.9985,0.9975,0.9840,0.9907,0.9717,0.9719,0.0200


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


5. Sélectionnez le meilleur modèle

In [7]:
qda_model = create_model("qda")
print(qda_model)
# svm = create_model("svm")
# print(svm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9917,0.9991,0.9950,0.9925,0.9938,0.9812,0.9812
1,0.9900,0.9990,0.9950,0.9900,0.9925,0.9774,0.9775
Mean,0.9908,0.9991,0.9950,0.9913,0.9931,0.9793,0.9794
Std,0.0008,0.0000,0.0000,0.0012,0.0006,0.0019,0.0019


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001)


6. Effectuez des prévisions sur vos données

In [8]:
predict_model(qda_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.9967,1.0000,1.0000,0.9950,0.9975,0.9925,0.9925


,diagonal,height_left,height_right,margin_low,margin_up,length,is_genuine,prediction_label,prediction_score
347,172.460007,103.779999,103.540001,3.89,3.20,113.050003,True,1,1.0000
451,172.169998,103.790001,103.540001,4.07,2.78,113.029999,True,1,1.0000
1028,171.740005,104.050003,103.930000,5.38,3.49,111.339996,False,0,1.0000
677,172.050003,103.900002,103.760002,4.52,2.71,113.419998,True,1,1.0000
690,172.330002,103.830002,103.540001,3.98,3.18,113.309998,True,1,1.0000
...,...,...,...,...,...,...,...,...,...
325,171.960007,104.169998,103.349998,4.35,3.31,113.669998,True,1,0.9999
507,171.970001,104.050003,104.110001,4.15,2.99,113.660004,True,1,0.9999
664,172.050003,103.699997,103.750000,5.04,2.27,113.550003,True,1,1.0000
415,172.000000,103.680000,104.099998,3.84,3.15,112.860001,True,1,0.9997


## Correction exercice SVM, K-NearestNeighbors, Naïve-Bayes et les méthodes Stochastiques

Correction exercice SVM (Titanic)

In [22]:
import pandas as pd
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

df = pd.read_csv('titanic.csv')
df = df[["pclass", "survived", "sex", "age", "fare", "embarked"]]
df = df.dropna()

gender_map = {'male' : 0, 'female': 1}
df["sex"] = df["sex"].map(gender_map)

X = df[["pclass", "sex", "age", "fare"]]
y = df.survived

svm_clf = svm.SVC(kernel='linear', C=100, class_weight='balanced')
# autre méthode
svm_model = LinearSVC()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=df["sex"], random_state=42)

svm_clf.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)

# Calcul de l'accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test,y_pred)
print("F1:", f1)
print("Accuracy:", accuracy)

F1: 0.7602339181286549
Accuracy: 0.8038277511961722


Suite aux résultats obtenus grâce à Pycaret, essayez d'améliorer l'Accuracy et le F1 score des SVM en modifiant certains paramètres.

In [10]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

data = pd.read_csv("billets.csv", sep=";")
data = data.dropna()

X = data.drop("is_genuine", axis=1)
y = data.is_genuine

svm_clf = svm.SVC(kernel='linear')
# autre méthode
svm_model = LinearSVC()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_clf.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = svm_model.predict(X_test)

# Calcul de l'accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test,y_pred)
print("F1:", f1)
print("Accuracy:", accuracy)

F1: 0.9845360824742267
Accuracy: 0.9795221843003413


Suite aux résultats obtenus grâce à Pycaret, essayez d'améliorer le score de K-NN en modifiant certains paramètres.

In [11]:
# Codez ici

Suite aux résultats obtenus grâce à Pycaret, essayez d'améliorer le score du modèle Naïve-Bayes en modifiant certains paramètres.

In [12]:
# Codez ici

Faites de même pour les méthodes Stochastiques.

In [13]:
# Codez ici

## En conclusion

 Bien que **Pycaret** vise à faciliter le processus de modélisation en automatisant certaines tâches, il **ne garantit pas toujours des résultats supérieurs**. Voici quelques raisons possibles pour cet écart de performances :

**- Recherche exhaustive des hyperparamètres** : PyCaret peut effectuer une recherche exhaustive des hyperparamètres, ce qui peut inclure des valeurs qui ne sont pas nécessairement optimales pour votre ensemble de données spécifique.
La recherche exhaustive peut conduire à des paramètres qui surajustent les données d'entraînement et n'obtiennent pas de bonnes performances sur des données inconnues.

**- Optimisation basée sur une métrique spécifique** : PyCaret utilise une métrique spécifique (par exemple, précision, rappel, AUC-ROC) pour optimiser les hyperparamètres. Si la métrique que vous avez utilisée pour évaluer votre modèle manuel diffère de celle utilisée par PyCaret, cela pourrait expliquer les écarts de performances.

**- Données d'entraînement spécifiques** : Il est possible que le modèle manuel ait été entraîné sur une division particulière des données, donnant des résultats spécifiques à cette division. PyCaret, en revanche, utilise souvent des techniques de validation croisée pour évaluer la performance du modèle de manière plus robuste.

**- Sensibilité aux données** : Certains algorithmes, y compris les k-NN, peuvent être sensibles aux caractéristiques spécifiques des données d'entraînement. De légères variations dans les données peuvent conduire à des performances différentes.